<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 0
fsdd_split = 0.3
wolfram_split = 0.3
experiment_id = '008'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

12

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def extract_mfcc(wave, sample_rate):
  mfccs = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc_ceptrum+2, n_mels=40, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [State( state_model, name=f"H{i}" ) for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals))]
  model.add_states(states)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataSet, model):
    if (len(testDataSet) == 0):
      return
    score_cnt = 0
    total = 0
    for label in testDataSet.keys():
        features = testDataSet[label]
        total += len(features)
        for sample in features:
            scoreList = {}
            for model_label in model.keys():
                score = model[model_label].log_probability(sample)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            if predict == label:
                score_cnt += 1
        print(f"{label}: ({score_cnt}/{total})")

    print("Final recognition rate is %.2f"%(100.0*score_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 4508053.123967816	Time (s): 0.2696
[2] Improvement: 14086.744048687979	Time (s): 0.2927
[3] Improvement: 4611.604927573586	Time (s): 0.2812
[4] Improvement: 2470.470764565165	Time (s): 0.2877
[5] Improvement: 1494.7924346631626	Time (s): 0.287
[6] Improvement: 1000.3837545479764	Time (s): 0.281
[7] Improvement: 683.5182868212578	Time (s): 0.2707
[8] Improvement: 455.4882526405854	Time (s): 0.282
[9] Improvement: 299.6052742723259	Time (s): 0.269
[10] Improvement: 216.5900207075756	Time (s): 0.2888
[11] Improvement: 180.9450653907843	Time (s): 0.2798
[12] Improvement: 121.74526260409039	Time (s): 0.2847
[13] Improvement: 56.671851940394845	Time (s): 0.2832
[14] Improvement: 30.834516364266165	Time (s): 0.2679
[15] Improvement: 18.139423976943363	Time (s): 0.2805
[16] Improvement: 11.571965110197198	Time (s): 0.2727
[17] Improvement: 7.748100577970035	Time (s): 0.2729
[18] Improvement: 5.240827410831116	Time (s): 0.2705
[19] Improvement: 3.6446799

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (129/837)
2: (321/1582)
6: (559/2428)
8: (680/3223)
0: (790/4018)
7: (983/4807)
9: (1274/5591)
5: (1701/6403)
3: (2087/7196)
4: (2545/7999)
Final recognition rate is 31.82 %
Evaluate on rest of FSDD set
0: (38/211)
1: (101/419)
3: (123/637)
9: (149/839)
7: (239/1073)
4: (373/1287)
5: (517/1490)
8: (566/1687)
2: (593/1901)
6: (624/2100)
Final recognition rate is 29.71 %
Evaluate on wolfram test set
4: (898/1655)
8: (1129/3315)
1: (1326/4940)
2: (1778/6654)
6: (2235/8278)
0: (2427/9948)
3: (3268/11593)
7: (3630/13247)
5: (4444/14889)
9: (5101/16567)
Final recognition rate is 30.79 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 2032932.7260411885	Time (s): 0.1577
[2] Improvement: 9348.194345248281	Time (s): 0.1506
[3] Improvement: 6540.546429567272	Time (s): 0.1615
[4] Improvement: 3430.170008165005	Time (s): 0.1637
[5] Improvement: 2344.484876073606	Time (s): 0.1631
[6] Improvement: 647.6218357039616	Time (s): 0.1492
[7] Improvement: 203.04969793831697	Time (s): 0.1603
[8] Improvement: 118.19972776400391	Time (s): 0.1489
[9] Improvement: 72.86453690740746	Time (s): 0.1668
[10] Improvement: 45.459553828579374	Time (s): 0.1553
[11] Improvement: 34.367599880322814	Time (s): 0.1554
[12] Improvement: 30.164624793513212	Time (s): 0.1471
[13] Improvement: 16.594975193496794	Time (s): 0.1575
[14] Improvement: 8.587619251862634	Time (s): 0.1491
[15] Improvement: 4.107587501581293	Time (s): 0.16
[16] Improvement: 1.9252695485483855	Time (s): 0.1725
[17] Improvement: 1.1517370219225995	Time (s): 0.1526
[18] Improvement: 0.8587415291694924	Time (s): 0.1494
[19] Improvement: 0.702

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (175/837)
2: (413/1582)
6: (619/2428)
8: (1006/3223)
0: (1184/4018)
7: (1238/4807)
9: (1391/5591)
5: (1638/6403)
3: (1870/7196)
4: (2108/7999)
Final recognition rate is 26.35 %
Evaluate on rest of FSDD set
0: (87/211)
1: (91/419)
3: (167/637)
9: (202/839)
7: (223/1073)
4: (330/1287)
5: (388/1490)
8: (495/1687)
2: (531/1901)
6: (543/2100)
Final recognition rate is 25.86 %
Evaluate on wolfram test set
4: (472/1655)
8: (1183/3315)
1: (1544/4940)
2: (2101/6654)
6: (2517/8278)
0: (2819/9948)
3: (3270/11593)
7: (3408/13247)
5: (3880/14889)
9: (4195/16567)
Final recognition rate is 25.32 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 874710.0100723584	Time (s): 0.1057
[2] Improvement: 1635.1734313009074	Time (s): 0.1098
[3] Improvement: 950.8451132802875	Time (s): 0.1114
[4] Improvement: 1166.0768486061133	Time (s): 0.1103
[5] Improvement: 3312.4252968347864	Time (s): 0.1081
[6] Improvement: 3355.0023974539945	Time (s): 0.1073
[7] Improvement: 1606.688702242158	Time (s): 0.124
[8] Improvement: 699.9610122453887	Time (s): 0.1155
[9] Improvement: 386.34939126414247	Time (s): 0.1164
[10] Improvement: 300.2508015263011	Time (s): 0.1049
[11] Improvement: 430.6507195729064	Time (s): 0.1128
[12] Improvement: 613.1203722801874	Time (s): 0.1096
[13] Improvement: 819.6626362093957	Time (s): 0.1085
[14] Improvement: 1261.531763857347	Time (s): 0.1093
[15] Improvement: 1241.4149454777944	Time (s): 0.1088
[16] Improvement: 623.1551753139938	Time (s): 0.118
[17] Improvement: 224.33441586408298	Time (s): 0.1182
[18] Improvement: 76.92575790715637	Time (s): 0.105
[19] Improvement: 33.889555

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (145/837)
2: (167/1582)
6: (274/2428)
8: (567/3223)
0: (904/4018)
7: (931/4807)
9: (1233/5591)
5: (1464/6403)
3: (1595/7196)
4: (1722/7999)
Final recognition rate is 21.53 %
Evaluate on rest of FSDD set
0: (133/211)
1: (196/419)
3: (241/637)
9: (278/839)
7: (283/1073)
4: (307/1287)
5: (353/1490)
8: (381/1687)
2: (388/1901)
6: (395/2100)
Final recognition rate is 18.81 %
Evaluate on wolfram test set
4: (258/1655)
8: (882/3315)
1: (1162/4940)
2: (1212/6654)
6: (1406/8278)
0: (2055/9948)
3: (2282/11593)
7: (2329/13247)
5: (2831/14889)
9: (3539/16567)
Final recognition rate is 21.36 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()